In [ ]:
import sys
import os
import pandas as pd
import numpy as np


ela_from_source = False
ela_from_source = True

if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\Users\SUD011\Documents\pyela-sudhir'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/github_jm/pyela')
    pkg_src_dir = root_src_dir
    sys.path.insert(0, pkg_src_dir)

In [ ]:
pkg_src_dir = "/home/per202/src/ela/pyvista"
sys.path.insert(0, pkg_src_dir)

In [ ]:
import pyvista as pv
pv.__version__

In [ ]:
#import panel
#import PVGeo    
#panel.extension("vtk")
#pv.set_plot_theme("document")
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'


## Global settings and preprocessed data


In [ ]:
import pyvista as pv
import pandas as pd
import numpy as np
import os
import sys

from ela.visual import discrete_classes_colormap
from pyvista_sample.VisualizeDataProcess import VisualizeDataProcess

if ('ELA_DATA' in os.environ):
    data_path = os.environ['ELA_DATA']
elif sys.platform == 'win32':
    data_path = r'C:\data\Lithology'
else:
    username = os.environ['USER']
    data_path = os.path.join('/home', username, 'data', 'Lithology')

In [ ]:
import pickle
cbr_datadir_out = os.path.join(data_path, 'Canberra','out')
drill_data_path = os.path.join(cbr_datadir_out, 'classified_logs.pkl')
dem_data_path = os.path.join(cbr_datadir_out,'dem_array_data.pkl')

interp_litho_filename = os.path.join(cbr_datadir_out,'3d_primary_litho.pkl')
with open(interp_litho_filename, 'rb') as handle:
    lithology_3d_array = pickle.load(handle)

# drill_data_path = r"C:\Users\Dennis.H\Desktop\CSIRO_data\cbr\classified_logs.pkl"
# dem_data_path = r"C:\Users\Dennis.H\Desktop\CSIRO_data\cbr\dem_array_data.pkl"

In [ ]:
dp = VisualizeDataProcess()
drill_data = dp.drill_file_read(drill_data_path)

# A convoluted way to remove nans
# vlah = {x for x in drill_data.Lithology_1_num.values if x==x}

# drill_data = drill_data[ drill_data.BoreID == 80000156 ]
# drill_data = drill_data[ drill_data.Lithology_1_num == 10.0 ]

dem_data = dp.dem_file_read(dem_data_path)
lines_dict = dp.drill_data_process(drill_data, 25, min_tube_radius = 70)
# temp = dp.drill_file_read(drill_data_path)
# pd.set_option('display.max_columns', None)
# print(temp)

grid = dp.dem_data_process(dem_data, 25)

layer = dp.lithology_layer_process(drill_data, dem_data, 'cbr', 25, 7, 10)

annotations = {
    00.0: 'shale',
    01.0: 'clay',
    02.0: 'granite',
    03.0: 'soil',
    04.0: 'sand',
    05.0: 'porphyry',
    06.0: 'siltstone',
    07.0: 'dacite',
    08.0: 'rhyodacite',
    09.0: 'gravel',
    10.0: 'limestone',
    11.0: 'sandstone',
    12.0: 'slate',
    13.0: 'mudstone',
    14.0: 'rock',
    15.0: 'ignimbrite',
    16.0: 'tuff'
}

sargs = dict(
    n_labels=len(annotations),
    bold=False,
    interactive=False,
    label_font_size=8,
    fmt="%.1f",
    font_family="arial",
    vertical=True,
    position_x=1,
    position_y=0.45,
    height=0.5,
)

In [ ]:
plotter = pv.Plotter()

for well in lines_dict.keys():
    plotter.add_mesh(lines_dict.get(well),
                     scalars=dp.scalar_prop_name,
                     scalar_bar_args=sargs,
                     annotations=annotations,
                     show_edges=False,
                     edge_color="white",
                     n_colors=len(annotations),
                     nan_color="black",
                     clim=[0, len(annotations)-1],
                     opacity=1,
                     )

# plotter.add_mesh(layer, scalars="Lithology", n_colors=len(annotations), clim=[0, len(annotations)-1], show_scalar_bar=False)
plotter.add_mesh(grid, opacity=0.9)
plotter.show_bounds(grid, show_xaxis=True, show_yaxis=True, show_zaxis=False)
plotter.show_axes()

In [ ]:
plotter.show()

In [ ]:
dem_data

In [ ]:
x_min, x_max, y_min, y_max =  dem_data['bounds']

In [ ]:
grid_res = dem_data['grid_res']

In [ ]:
# Create the 3D NumPy array of spatially referenced data
# This is spatially referenced such that the grid is 20 by 5 by 10
#   (nx by ny by nz)
values = dem_data['dem_array']

dim_x, dim_y = values.shape

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values on the
#   POINT data
grid.dimensions = (dim_x, dim_y, 1)

# Edit the spatial reference
grid.origin = (x_min, y_min, 0)  # The bottom left corner of the data set
grid.spacing = (grid_res,grid_res, 1)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.point_arrays["values"] = values.flatten(order="F")  # Flatten the array!

In [ ]:
grid = grid.warp_by_scalar()

In [ ]:
from itkwidgets import view

In [ ]:
view(geometries=grid)

In [ ]:
poly_data = lines_dict['80000097']

In [ ]:
poly_data

In [ ]:
bores = [ p for p in lines_dict.values() ]

In [ ]:
view(geometries = bores[0:4])

In [ ]:
dp = VisualizeDataProcess()

In [ ]:
surf_data = dp.dem_data_process(dem_data, 20)

In [ ]:
view(geometries=([surf_data] + bores[0:4]))

In [ ]:
# TODO: how could this information be serialised/reloaded between notebooks?
# (442.0, 824.0)
dp.ahd_min=442
dp.ahd_max=824
ahd_min=442
ahd_max=824
z_ahd_coords = np.arange(ahd_min,ahd_max,1)
dim_x,dim_y,dim_z = lithology_3d_array.shape
dims = (dim_x,dim_y,dim_z)

In [ ]:
height_adjustment_factor = 20
layer_from = 2
layer_to=3
vol = dp.build_layer_mesh(lithology_3d_array, height_adjustment_factor, layer_from, layer_to)

In [ ]:
view(geometries=[vol, surf_data])

In [ ]:
vol.point_arrays

In [ ]:
vol = grid

In [ ]:
type(vol), vol.dimensions

In [ ]:
viewer = view(geometries=vol)

In [ ]:
# `view` returns the widget object
viewer = view(image, annotations=False)

# Create an itk smoother filter object. By re-using the object, output memory-reallocation is avoided
smoother = itk.MedianImageFilter.New(image)

# Define a function to use with ipywidgets `interactive`
def smooth_and_view(radius=2):
    smoother.SetRadius(radius)
    smoother.Update()
    # Update the image used in the viewer
    viewer.image = smoother.GetOutput()
slider = interactive(smooth_and_view, radius=(0, 10, 1))

widgets.VBox([viewer, slider])

## Experimenting with pyvista's PlotterITK.

In [ ]:
plotter = pv.PlotterITK()

In [ ]:
for well in lines_dict.keys():
    plotter.add_mesh(lines_dict.get(well),
                     scalars=dp.scalar_prop_name,
                     scalar_bar_args=sargs,
                     annotations=annotations,
                     show_edges=False,
                     edge_color="white",
                     n_colors=len(annotations),
                     nan_color="black",
                     clim=[0, len(annotations)-1],
                     opacity=1,
                     )

# plotter.add_mesh(layer, scalars="Lithology", n_colors=len(annotations), clim=[0, len(annotations)-1], show_scalar_bar=False)
plotter.add_mesh(grid, opacity=0.9)
#plotter.show_bounds(grid, show_xaxis=True, show_yaxis=True, show_zaxis=False)
#plotter.show_axes()

CAUTION: the following as of 2019-10-14 will crash the browser tab.

In [ ]:
# Crashes the browser tab as of 2019-10-06
plotter.show()